# Pokemon Price Predictor Using Closing Prices Dataset

In this notebook, we will build and train a custom LSTM RNN that uses a 4 year window of Pokemon Boxset closing prices to predict the next month's closing price. 

1. Prepare the data for training and testing
2. Build and train a custom LSTM RNN
3. Evaluate the performance of the model

## Data Preparation

In this section, you will need to prepare the training and testing data for the model. The model will use a rolling 10 day window to predict the 11th day closing price.

You will need to:
1. Use the `window_data` function to generate the X and y values for the model.
2. Split the data into 70% training and 30% testing
3. Apply the MinMaxScaler to the X and y values
4. Reshape the X_train and X_test data for the model. Note: The required input format for the LSTM is:

```python
reshape((X_train.shape[0], X_train.shape[1], 1))
```

In [9]:
import numpy as np
import pandas as pd
import hvplot.pandas

In [10]:
# Set the random seed for reproducibility
# Note: This is for the homework solution, but it is good practice to comment this out and run multiple experiments to evaluate your model
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [11]:
# Load the historical closing prices for Pokemon Dataset
df = pd.read_csv('pokemon_cardset.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)
df = df.sort_index()
df.head()

,Base Set 1st Ed.,Base Set 1st Ed. Holos,Base Set Shadowless,Base Set Shadowless Holos,Base Set Unlimited,Base Set Unlimited Holos,Jungle 1st Edition,Jungle 1st Ed. Holo,Jungle Unlimited,Jungle Unlimited Holos,...,EX Legend Maker EX Holos Only,EX Holon Phantoms Holos Exc. Reverses,EX Holon Phantoms EX Holos Only,EX Crystal Guardians Holos Exc. Reverses,EX Crystal Guardians EX Holos Only,EX Dragon Frontiers Holos Exc. Reverses,EX Dragon Frontiers EX Holos Only,EX Power Keepers Holos Exc. Reverses,EX Power Keepers EX Holos Only,EX Set Gold Star Holos
Date,,,,,,,,,,,,,,,,,,,,,
2016-10-01,3788.38,2858.07,369.87,369.87,459.94,454.94,510.21,262.05,51.00,41.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,298.45
2016-11-01,6636.07,5558.35,951.71,933.20,544.30,529.29,881.14,479.17,76.45,66.45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1658.19
2016-12-01,7437.45,6295.23,1466.50,1397.01,486.49,415.98,1009.54,534.77,98.95,81.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1786.44
2017-01-01,6565.65,5351.51,1487.78,1416.03,540.26,468.75,1062.67,600.18,100.59,83.59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1967.91
2017-02-01,6788.13,5605.40,1548.22,1471.97,578.97,510.95,990.79,514.48,140.13,94.09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2369.52


In [12]:
# This function accepts the column number for the features (X) and the target (y)
# It chunks the data up with a rolling window of Xt-n to predict Xt
# It returns a numpy array of X any y
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [13]:
# Predict Closing Prices using a 30 month window of previous closing prices
window_size = 30

# Column index 0 is the 'Base Set 1st Ed.' column
# Column index 1 is the `Base Set 1st Ed. Holos` column
feature_column = 0
target_column = 0
X, y = window_data(df, window_size, feature_column, target_column)

In [14]:
# Use 70% of the data for training and the remaindder for testing
split = int(0.7 * len(X))
X_train = X[: split - 1]
X_test = X[split:]
y_train = y[: split -1]
y_test = y[split:]

In [15]:
from sklearn.preprocessing import MinMaxScaler
# Use the MinMaxScaler to scale data between 0 and 0.
scaler = MinMaxScaler()
scaler.fit(X)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

scaler.fit(y)
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)

In [16]:
# Reshape the features for the model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

---

## Build and Train the LSTM RNN

In this section, you will design a custom LSTM RNN and fit (train) it using the training data.

You will need to:
1. Define the model architecture
2. Compile the model
3. Fit the model to the training data

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [20]:
# Build the LSTM model. 
# The return sequences need to be set to True if you are adding additional LSTM layers, but 
# You don't have to do this for the final layer. 
# Note: The dropouts help prevent overfitting
# Note: The input shape is the number of time steps and the number of indicators
# Note: Batching inputs has a different input shape of Samples/TimeSteps/Features

model = Sequential()

number_units = 30
dropout_fraction = 0.20

# Layer 1
model.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train.shape[1], 1))
    )
model.add(Dropout(dropout_fraction))

# Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))

# Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))

# Output layer
model.add(Dense(1))

In [21]:
# Compile the model
model.compile(optimizer="adam", loss="mean_squared_error")

In [22]:
# Summarize the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 30, 30)            3840      
_________________________________________________________________
dropout_3 (Dropout)          (None, 30, 30)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 30, 30)            7320      
_________________________________________________________________
dropout_4 (Dropout)          (None, 30, 30)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 30)                7320      
_________________________________________________________________
dropout_5 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                

In [23]:
# Train the model
# Use at least 10 epochs
# Do not shuffle the data
# Experiement with the batch size, but a smaller batch size is recommended
epochs = 10
batch_size = 5
model.fit(X_train, y_train, epochs=epochs, shuffle=False, batch_size=batch_size, verbose=1)

Epoch 1/10
3/3 [==============================] - 0s 24ms/step - loss: 1.7258e-04
Epoch 2/10
3/3 [==============================] - 0s 24ms/step - loss: 0.0017
Epoch 3/10
3/3 [==============================] - 0s 25ms/step - loss: 9.4295e-04
Epoch 4/10
3/3 [==============================] - 0s 23ms/step - loss: 7.2218e-04
Epoch 5/10
3/3 [==============================] - 0s 25ms/step - loss: 4.2522e-04
Epoch 6/10
3/3 [==============================] - 0s 25ms/step - loss: 4.6143e-04
Epoch 7/10
3/3 [==============================] - 0s 24ms/step - loss: 6.3237e-04
Epoch 8/10
3/3 [==============================] - 0s 22ms/step - loss: 7.8367e-04
Epoch 9/10
3/3 [==============================] - 0s 25ms/step - loss: 5.7308e-04
Epoch 10/10
3/3 [==============================] - 0s 24ms/step - loss: 5.3095e-04


---

## Model Performance

In this section, you will evaluate the model using the test data. 

You will need to:
1. Evaluate the model using the `X_test` and `y_test` data.
2. Use the X_test data to make predictions
3. Create a DataFrame of Real (y_test) vs predicted values. 
4. Plot the Real vs predicted values as a line chart

### Hints
Remember to apply the `inverse_transform` function to the predicted and y_test values to recover the actual closing prices.

In [24]:
# Evaluate the model
model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 2ms/step - loss: 0.2482


0.24824224412441254

In [25]:
# Make some predictions
predicted = model.predict(X_test)

In [26]:
# Recover the original prices instead of the scaled version
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

In [27]:
# Create a DataFrame of Real and Predicted values
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
}, index = df.index[-len(real_prices): ]) 
stocks.head()

,Real,Predicted
Date,,
2020-07-01,35770.11,16871.812500
2020-08-01,41624.22,17032.800781
2020-09-01,53946.81,17249.064453
2020-10-01,68546.13,17530.156250
2020-11-01,159305.39,17938.128906


In [28]:
# Plot the real vs predicted values as a line chart
stocks.hvplot.line(xlabel="Date",
                  ylabel="Price",
                  title="LSTM Cardset Model: Real vs Predicted Values")

:NdOverlay   [Variable]
   :Curve   [Date]   (value)